In [38]:
import pandas as pd
import sqlite3

In [39]:
# criando um dataframe
dados = pd.DataFrame ({'nome':['Júlia', 'America', 'Oceane'],
                       'idade':[19,44,29],
                       'cargo': ['Engenheira de dados', 'Cientista de dados','Analista de dados']
                       
})

In [40]:
dados

,nome,idade,cargo
0,Júlia,19,Engenheira de dados
1,America,44,Cientista de dados
2,Oceane,29,Analista de dados


In [41]:
import os
os.makedirs('JupyterLab/database1', exist_ok=True)

In [42]:
import os
print(os.getcwd())  # Mostra o diretório atual do Jupyter

C:\Users\mnasc\Desktop\JupyterLab


In [43]:
import sqlite3
cnn = sqlite3.connect('database/dbprojeto2.db')  # Remova "JupyterLab/" do caminho
print("Banco de dados conectado com sucesso!")

Banco de dados conectado com sucesso!


In [45]:
dados.to_sql('funcionarios', cnn)

In [46]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [47]:
%sql sqlite:///database/dbprojeto2.db

In [48]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [49]:
%%sql 
SELECT count (*) FROM funcionarios

 * sqlite:///database/dbprojeto2.db
Done.


count (*)
3


In [50]:
#Calculando a média de idade dos funcionarios

In [51]:
%%sql

SELECT avg(idade) as 'idade_media' FROM funcionarios

 * sqlite:///database/dbprojeto2.db
Done.


idade_media
30.666666666666668


# Problema
Temos aqui um arquivo com dados de pacientes que desenvolveram ou não diabetes. Precisamos gerar uma amostra de dados com os pacientes com mais de 50 anos e para cada um deles indicar uma nova coluna se o paciente está normal (IMC menor que 30) ou obeso (IMC maior ou igual 30). Então, devemos gerar um arquivo CSV e encaminhar para o Cientista de dados.

In [ ]:
# primeiramente carregamos o arquivo original

In [53]:
df = pd.read_csv('dataset/diabetes.csv')

In [54]:
df.shape

(768, 9)

In [55]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [56]:
#copiando o dataframe para dentro do banco de dados como uma tabela

In [57]:
df.to_sql('diabetes', cnn)

768

In [59]:
%%sql

SELECT count (*) FROM diabetes

 * sqlite:///database/dbprojeto2.db
Done.


count (*)
768


In [ ]:
# A coluna 'Outcome' retorna se o paciente desenvolveu ou não diabetes.

In [60]:
%%sql 

SELECT age, Glucose, Outcome FROM diabetes WHERE Glucose > 190

 * sqlite:///database/dbprojeto2.db
Done.


Age,Glucose,Outcome
53,197,1
41,196,1
41,194,1
57,196,1
31,197,0
24,193,0
34,191,0
59,194,1
29,196,1
25,193,1


In [ ]:
# Verificando as colunas que tenho no DataFrame

In [61]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [62]:
# Agora vamos criar uma tabela nova no banco de dados
# BMI DECIMAL (8, 2) = OITO POSIÇÕES E DUAS CASAS DECIMAIS

In [63]:
%%sql

CREATE TABLE pacientes (Pregnancies INT, 
                        Glucose INT, 
                        BloodPressure INT,
                        SkinThicknessINT,
                        Insulin INT,
                        BMI DECIMAL (8, 2),
                        DiabetesPedigreeFunction DECIMAL (8,2),
                        Age INT,
                        Outcome INT);

 * sqlite:///database/dbprojeto2.db
Done.


[]

In [64]:
%%sql

SELECT * FROM pacientes

 * sqlite:///database/dbprojeto2.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThicknessINT,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


In [66]:
%%sql

INSERT INTO pacientes (Pregnancies, 
                        Glucose, 
                        BloodPressure,
                        SkinThicknessINT,
                        Insulin,
                        BMI,
                        DiabetesPedigreeFunction,
                        Age,
                        Outcome)
SELECT Pregnancies, 
        Glucose, 
        BloodPressure,
        SkinThickness,
        Insulin,
        BMI,
        DiabetesPedigreeFunction,
        Age,
        Outcome
FROM diabetes WHERE Age >50;

 * sqlite:///database/dbprojeto2.db
81 rows affected.


[]

In [67]:
%%sql 

SELECT * FROM pacientes

 * sqlite:///database/dbprojeto2.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThicknessINT,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
2,197,70,45,543,30.5,0.158,53,1
8,125,96,0,0,0,0.232,54,1
10,139,80,0,0,27.1,1.441,57,0
1,189,60,23,846,30.1,0.398,59,1
5,166,72,19,175,25.8,0.587,51,1
11,143,94,33,146,36.6,0.254,51,1
13,145,82,19,110,22.2,0.245,57,0
5,109,75,26,0,36,0.546,60,0
4,111,72,47,207,37.1,1.39,56,1
9,171,110,24,240,45.4,0.721,54,1


### Aqui já criamos o primeiro requerimento, que são pacientes com mais de 50 anos de idade.
Agora nós vamos indicar se o paciente está normal ou obeso, com base no IMC/BMI (indice de massa corporal).
Vamos criar uma nova coluna na nossa tabela de pacientes.

In [69]:
%%sql

ALTER TABLE pacientes 
ADD perfil VARCHAR (10);

 * sqlite:///database/dbprojeto2.db
Done.


[]

In [70]:
%%sql 

SELECT * FROM pacientes

 * sqlite:///database/dbprojeto2.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThicknessINT,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,perfil
2,197,70,45,543,30.5,0.158,53,1,None
8,125,96,0,0,0,0.232,54,1,None
10,139,80,0,0,27.1,1.441,57,0,None
1,189,60,23,846,30.1,0.398,59,1,None
5,166,72,19,175,25.8,0.587,51,1,None
11,143,94,33,146,36.6,0.254,51,1,None
13,145,82,19,110,22.2,0.245,57,0,None
5,109,75,26,0,36,0.546,60,0,None
4,111,72,47,207,37.1,1.39,56,1,None
9,171,110,24,240,45.4,0.721,54,1,None


In [72]:
%%sql

UPDATE pacientes
SET perfil = 'Normal'
WHERE BMI < 30;

 * sqlite:///database/dbprojeto2.db
38 rows affected.


[]

In [73]:
%%sql

SELECT * FROM pacientes

 * sqlite:///database/dbprojeto2.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThicknessINT,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,perfil
2,197,70,45,543,30.5,0.158,53,1,None
8,125,96,0,0,0,0.232,54,1,Normal
10,139,80,0,0,27.1,1.441,57,0,Normal
1,189,60,23,846,30.1,0.398,59,1,None
5,166,72,19,175,25.8,0.587,51,1,Normal
11,143,94,33,146,36.6,0.254,51,1,None
13,145,82,19,110,22.2,0.245,57,0,Normal
5,109,75,26,0,36,0.546,60,0,None
4,111,72,47,207,37.1,1.39,56,1,None
9,171,110,24,240,45.4,0.721,54,1,None


In [74]:
%%sql

UPDATE pacientes
SET perfil = 'Obeso'
WHERE BMI >= 30;

 * sqlite:///database/dbprojeto2.db
43 rows affected.


[]

In [75]:
%%sql

SELECT * FROM pacientes

 * sqlite:///database/dbprojeto2.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThicknessINT,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,perfil
2,197,70,45,543,30.5,0.158,53,1,Obeso
8,125,96,0,0,0,0.232,54,1,Normal
10,139,80,0,0,27.1,1.441,57,0,Normal
1,189,60,23,846,30.1,0.398,59,1,Obeso
5,166,72,19,175,25.8,0.587,51,1,Normal
11,143,94,33,146,36.6,0.254,51,1,Obeso
13,145,82,19,110,22.2,0.245,57,0,Normal
5,109,75,26,0,36,0.546,60,0,Obeso
4,111,72,47,207,37.1,1.39,56,1,Obeso
9,171,110,24,240,45.4,0.721,54,1,Obeso


### Carregando os Dados no Pandas e salvando o CSV

In [76]:
#Query 
query = cnn.execute('SELECT * FROM pacientes')
query

In [77]:
# List comprehension
cols = [coluna[0] for coluna in query.description]
cols

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThicknessINT',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome',
 'perfil']

In [79]:
# Gerando o dataframe 
resultado = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [80]:
resultado.shape

(81, 10)

In [82]:
resultado.head()

,Pregnancies,Glucose,BloodPressure,SkinThicknessINT,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,perfil
0,2,197,70,45,543,30.5,0.158,53,1,Obeso
1,8,125,96,0,0,0.0,0.232,54,1,Normal
2,10,139,80,0,0,27.1,1.441,57,0,Normal
3,1,189,60,23,846,30.1,0.398,59,1,Obeso
4,5,166,72,19,175,25.8,0.587,51,1,Normal


In [83]:
#Salvando o arquivo CSV
resultado.to_csv('dataset/pacientes.csv', index = False)

# Fim